# Load/Dump Arrays And Sessions From/To Files


Import the LArray library:


In [ ]:
from larray import *

## Load from CVS, Excel or HDF5 files

Arrays can be loaded from CSV files

```python
# read_tsv is a shortcut when data are separated by tabs instead of commas (default separator of read_csv)
# read_eurostat is a shortcut to read EUROSTAT TSV files
household = read_csv('hh.csv')
```


or Excel sheets

```python
# loads array from the first sheet if no sheet is given
pop = read_excel('demography.xlsx', 'pop')
```


or HDF5 files (HDF5 is file format designed to store and organize large amounts of data.
An HDF5 file can contain multiple arrays.

```python
mortality = read_hdf('demography.h5','qx')
```


See documentation of reading functions for more details.


### Load Sessions


The advantage of sessions is that you can load many arrays in one shot:

```python
# this load several arrays from a single Excel file (each array is stored on a different sheet)
s = Session()
s.load('test.xlsx')
# or 
s = Session('test.xlsx')

# this load several arrays from a single HDF5 file (which is a very fast format)
s = Session()
s.load('test.h5')
# or 
s = Session('test.h5')
```


## Dump to CSV, Excel or HDF5 files

Arrays can be dumped in CSV files

```python
household.to_csv('hh2.csv')
```


or in Excel files

```python
# if the file does not already exist, it is created with a single sheet,
# otherwise a new sheet is added to it
household.to_excel('demography_2.xlsx', overwrite_file=True)
# it is usually better to specify the sheet explicitly (by name or position) though
household.to_excel('demography_2.xlsx', 'hh')
```


or in HDF5 files

```python
household.to_hdf('demography_2.h5', 'hh')
```


See documentation of writing methods for more details.


### Dump Sessions


The advantage of sessions is that you can save many arrays in one shot:

```python
# this saves all the arrays in a single excel file (each array on a different sheet)
s.save('test.xlsx')

# this saves all the arrays in a single HDF5 file (which is a very fast format)
s.save('test.h5')
```


## Interact with Excel files


### Write Arrays

Open an Excel file

```python
wb = open_excel('test.xlsx', overwrite_file=True)
```


Put an array in an Excel Sheet, **excluding** headers (labels)

```python
# put arr at A1 in Sheet1, excluding headers (labels)
wb['Sheet1'] = arr
# same but starting at A9
# note that Sheet1 must exist
wb['Sheet1']['A9'] = arr
```


Put an array in an Excel Sheet, **including** headers (labels)

```python
# dump arr at A1 in Sheet2, including headers (labels)
wb['Sheet2'] = arr.dump()
# same but starting at A10
wb['Sheet2']['A10'] = arr.dump()
```


Save file to disk

```python
wb.save()
```


Close file

```python
wb.close()
```


### Read Arrays

Open an Excel file

```python
wb = open_excel('test.xlsx')
```


Load an array from a sheet (assuming the presence of (correctly formatted) headers and only one array in sheet)

```python
# save one array in Sheet3 (including headers)
wb['Sheet3'] = arr.dump()

# load array from the data starting at A1 in Sheet3
arr = wb['Sheet3'].load()
```


Load an array with its axes information from a range

```python
# if you need to use the same sheet several times,
# you can create a sheet variable
sheet2 = wb['Sheet2']

# load array contained in the 4 x 4 table defined by cells A10 and D14
arr2 = sheet2['A10:D14'].load()
```


### Read Ranges (experimental)

Load an array (raw data) with no axis information from a range

```python
arr3 = wb['Sheet1']['A1:B4']
```


in fact, this is not really an LArray ...

```python
type(arr3)

larray.io.excel.Range
```


... but it can be used as such

```python
arr3.sum(axis=0)
```


... and it can be used for other stuff, like setting the formula instead of the value:

```python
arr3.formula = '=D10+1'
```


In the future, we should also be able to set font name, size, style, etc.
